In [1]:
#get tokenizer

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)


In [2]:
from transformers import pipeline

model_trained = pipeline('text-generation',model='./results', tokenizer=tokenizer, framework = "pt")

#model_before_train = pipeline('text-generation',model = 'gpt2', tokenizer = tokenizer)

In [4]:
#test the model :)
def gen_text(text, tokens_per_gen = 50, generator = model_trained):
  encoded_input = tokenizer(text, add_special_tokens=True, return_tensors="pt")
  length = len(encoded_input["input_ids"][0])
  return generator(text, max_length = length + tokens_per_gen , num_return_sequence = 1, no_repeat_ngram_size = 2, early_stopping = True, skip_special_tokens=False, return_full_text=False)[0]['generated_text']



In [5]:
def cut_uncomplete_sentence(text):
    index = text.rfind('.')
    return text[:index+1]

In [10]:
import tkinter as tk

text_list = []
init_text = "You wake up and there is something in front of you. "
text_list.append(init_text)
text = gen_text(init_text, tokens_per_gen=60)
text = cut_uncomplete_sentence(text)
text_list.append(text)


window = tk.Tk()
window.title("DungeonAI")
window.geometry("600x600")
text_box = tk.Text()
text_box.pack(expand=True, fill='both')
text_box.insert(tk.END, init_text)
text_box.insert(tk.END, text + '\n' + '\n')

#doesn't work yet
def click_enter(event):
    global text_list
    if len(text_list) == 5:
        del text_list[0]
    #get text in gui - generate more
    text = text_box.get("1.0", tk.END)
    new_text = gen_text(' '.join(text_list))
    #clean text
    new_text = cut_uncomplete_sentence(new_text)
    #append to text_list
    text_list.append(new_text)
    text_box.insert(tk.END, new_text + '\n'+'\n')

window.bind('<Return>', click_enter)
window.mainloop()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
